In [1]:
import requests
import lxml.html as lh
import pandas as pd
import re

In [2]:
URL_Dict = {'Liquid Funds':'https://cleartax.in/s/best-liquid-funds',
            'Short term Mutual Funds':'https://cleartax.in/s/best-short-term-mutual-funds',
            'Arbitrage Funds':'https://cleartax.in/s/arbitrage-funds'}

In [3]:

def get_fund_details(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    return_and_nav = doc.xpath('//ul/li/span[@class="value__nkb-6"]')
    return_avg = return_and_nav[0].text
    net_asset = return_and_nav[1].text
    returns_per_yr.append(return_avg)
    NAV.append(net_asset)

In [4]:
def get_funds(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    tr_elements = doc.xpath('//tr')
    col = []
    i = 0
    
    for t in tr_elements[0]:
        i+=1
        name = t.text_content()
        col.append((name,[]))
    
    for j in range(1,len(tr_elements)):
        T = tr_elements[j]
        if len(T)!=3:
            break
        i = 0
        for t in T.iterchildren():
            data = t.text_content()
            if i>0:
                try:
                    data = int(data)
                except:
                    pass
            col[i][1].append(data)
            i+=1
    
    Dict = {title:column for (title,column) in col}
    df = pd.DataFrame(Dict)
    df = df.replace('\r',"",regex=True)
    df = df.replace('\n',"",regex=True)
    heading = []
    for head in df.columns.tolist():
        data = re.sub('\n',"",head)
        data = re.sub('\r',"",data)
        heading.append(data)
    df.columns = heading
    df = df.drop('Link',axis = 1)
    
    link = doc.xpath('//tr/td/p/a[@href]/@href')
    link = list(dict.fromkeys(link))
    df['Link'] = link
    
    
    for l in df['Link']:
        get_fund_details(l)
    
    df['Returns Per Year (5 year avg)'] = returns_per_yr
    df['Net asset value'] = NAV
    
    df.to_csv(r'C:\Users\jsabh\OneDrive\Desktop\finastra'+ '\\' + investment +'.csv',index = False)

In [5]:
for url in URL_Dict:
    investment = url
    returns_per_yr = []
    NAV = []
    get_funds(URL_Dict[url])